In [1]:
import json
import openai

openai.api_base = "http://localhost:8000"
openai.api_key = "test"

In [2]:
# Define functions
from datetime import datetime

def get_time(location, format="12-hour"):
    times = {"Singapore": "08:00", "London": "00:00", "New York": "20:00", "Beijing": "08:00"}
    return datetime.strptime(times[location], "%H:%M").strftime("%r") if format == "12-hour" else times[location]

In [3]:
# Call the model the first time with the functions
messages = [{"role": "user", "content": "Can you check what is the time in Singapore?"}]
response = openai.ChatCompletion.create(
    model="jeffrey-fong/invoker-13b",
    messages=messages,
    functions=[
        {
            "name": "get_time",
            "description": "Get the current time",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. New York City, NY"
                    },
                    "format": {
                    "type": "string",
                    "enum": ["12-hour", "24-hour"]
                    }
                },
                "required": ["location"]
            }
        }
    ]
)
response

<OpenAIObject chat.completion id=e2322c58-6aa6-4539-8cbc-e5219b67befe at 0x7f2bc8256890> JSON: {
  "id": "e2322c58-6aa6-4539-8cbc-e5219b67befe",
  "object": "chat.completion",
  "created": 1695652179,
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": null,
        "name": null,
        "function_call": {
          "name": "get_time",
          "arguments": "{\"location\": \"Singapore\", \"format\": \"12-hour\"}"
        }
      },
      "finish_reason": "function_call"
    }
  ]
}

In [8]:
response_message = response["choices"][0]["message"]
# Call the function indicated by the model if the model generates a function call
# Append the messages and call the model again to get a summarized assistant response
if response_message.get("function_call"):
    available_functions = {"get_time": get_time}
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(
        location=function_args.get("location"),
        format=function_args.get("format"),
    )
    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = openai.ChatCompletion.create(
        model="jeffrey-fong/invoker-13b",
        messages=messages,
    )
second_response

<OpenAIObject chat.completion id=912bec9a-d3c6-473f-8161-702b45d393e2 at 0x7f2ba4553c90> JSON: {
  "id": "912bec9a-d3c6-473f-8161-702b45d393e2",
  "object": "chat.completion",
  "created": 1695652744,
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "The time in Singapore is 08:00 AM.",
        "name": null,
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}